In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
import torch
import trimesh
from functools import partial
import ssl
from Solvers import SolverEmbedding,Models,Loss
from config import cfg
from dataEmbedding.dataEmbedding import Read_Load_BuildBatch
from dataEmbedding.dataEmbeddingLoader import GenerateDataLoader,check_dataset,collate_embedding
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device=torch.device("cuda")
else:
    device=torch.device("cpu")
import nltk
#nltk.download("stopwords")
#nltk.download('wordnet')
device=cfg.DEVICE
print(device)

#for mac os fix 
ssl._create_default_https_context = ssl._create_unverified_context

cpu


In [2]:
def vizualizer_stanford_data(vox_color):
    vox_color = np.rollaxis(vox_color, 0,4)
    ind=vox_color[:,:,:]!=[0,0,0,0]
    vox = ind[:, :, :,0]
    z=trimesh.voxel.VoxelGrid(vox)
    l=z.as_boxes(colors=vox_color)
    return l.show()

In [3]:
#primiData = Read_Load_BuildBatch('/primitives',batchSize,'primitives')
stanData=Read_Load_BuildBatch(cfg.EMBEDDING_VOXEL_FOLDER,cfg.EMBEDDING_BATCH_SIZE,'stanford_data',cfg.EMBEDDING_CAPTION_CSV)


In [5]:
train_dataset = GenerateDataLoader(stanData.data_agg_train,'stanford_data',stanData.data_dir,stanData.labelEnc,stanData.dict_word2idx,'train')

val_dataset = GenerateDataLoader(stanData.data_agg_val,'stanford_data',stanData.data_dir,stanData.labelEnc,stanData.dict_word2idx,'val')

test_dataset=GenerateDataLoader(stanData.data_agg_test,'stanford_data',stanData.data_dir,stanData.labelEnc,stanData.dict_word2idx,'test')

dataloader = {
            'train': DataLoader(
                train_dataset, 
                batch_size=cfg.EMBEDDING_BATCH_SIZE * 2,              
                drop_last=check_dataset(train_dataset, cfg.EMBEDDING_BATCH_SIZE * 2),
                collate_fn=collate_embedding,
                num_workers=4
            ),
            'val': DataLoader(
                val_dataset, 
                batch_size=cfg.EMBEDDING_BATCH_SIZE*2,
                collate_fn=collate_embedding,
                num_workers=4
            ),
            'test': DataLoader(
                test_dataset, 
                batch_size=cfg.EMBEDDING_BATCH_SIZE*2,
                collate_fn=collate_embedding
                #num_workers=2
            )
    }       


In [5]:
for iter,(_,_,labels,texts,_,shapes) in enumerate(dataloader['train']):
    
    batch_size = shapes.size(0)
    texts = texts.to(device)
    text_labels = labels.to(device)

    shapes = shapes.to(device).index_select(0, torch.LongTensor([i * 2 for i in range(batch_size // 2)]).to(device))
    shape_labels = labels.to(device).index_select(0, torch.LongTensor([i * 2 for i in range(batch_size // 2)]).to(device))
        

    #s = ShapeModel(shapes)
    #t = TextModel(texts)

    print(texts.shape)

    if iter==5:
      break

torch.Size([128, 10])
torch.Size([128, 10])
torch.Size([128, 10])
torch.Size([128, 10])
torch.Size([128, 10])
torch.Size([128, 10])


In [6]:
criterion={
        'walker': Loss.RoundTripLoss(device=device),
        'visit': Loss.AssociationLoss(device=device),
        'metric': Loss.SmoothedMetricLoss(device=device)
        }

In [7]:
ShapeModel=Models.ShapeEncoder()
ShapeModel=ShapeModel.to(device)
TextModel=Models.TextEncoder(len(stanData.dict_word2idx))
TextModel=TextModel.to(device)
optimizer = torch.optim.Adam(list(ShapeModel.parameters()) + list(TextModel.parameters()), lr=cfg.EMBEDDING_LR, weight_decay=cfg.EMBEDDING_WEIGHT_DC)
history=SolverEmbedding.Solver(TextModel,ShapeModel,dataloader,optimizer,criterion,cfg.EMBEDDING_BATCH_SIZE,device)

In [8]:
history.train(cfg.EMBEDDING_EPOCH_NR)

epoch [1/16] starting...



  0%|          | 1/980 [00:05<1:30:53,  5.57s/it]

91.42755889892578


  3%|▎         | 29/980 [00:23<10:41,  1.48it/s] 